In [28]:
import pandas as pd
import numpy as np
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
import jieba
from re import sub

In [29]:
#读取数据
train_data = pd.read_csv('sohu.csv')
train_data

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
5,pic,10月4日十佳图：李娜小德球迷抢眼 大威显文静责编:马克杰 日期:2013-10-4 小德的...
6,pic,高清：小德速胜高举双臂庆祝 鼓掌向球迷们致意责编:马克杰 日期:2013-10-4 小德鼓掌...
7,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
8,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
9,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."


In [33]:
#数据预处理
train_data['label']=train_data['label'].map({'pic': 0, 'news': 1, 'sports': 2, 'business':3, 'caipiao':4, 'yule':5, 'mil':6,
       'cul':7})
for i in train_data['text']:
    #去除文本中的标点符号
    i = re.sub(r"[\s+\·<>?《》“”.\!\/_,$%^*(【】：\]\[\-:;+\"\']+|[+——！，。？、~@#￥%……&*（）]+|[0-9]+","",i)
    #分词
    i = jieba.cut(i, cut_all = True)
train_data['text'] = train_data['text'].apply(lambda x:' '.join(jieba.cut(x)))
train_data

,label,text
0,NaN,高清 ： 彭帅 谢淑薇 遗憾 遭...
1,NaN,河南 周口 路边 秸秆 燃烧 ...
2,NaN,消息 称 138 名 中国 劳工...
3,NaN,越南 重要 军事 领导人 武元甲 ...
4,NaN,西沙 搜救 已 发现 6 具 ...
5,NaN,10 月 4 日 十佳 图 ...
6,NaN,高清 ： 小德 速胜 高举 双臂...
7,NaN,卡努特 伤愈 将 首发 战辽足 ...
8,NaN,美 “ 宙斯 盾 ” 系统 ...
9,NaN,3 中国 建筑 工 俄罗斯 身亡...


In [25]:
#提取特征
from keras.preprocessing.text import Tokenizer
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train_data['text'])
sequences = tokenizer.texts_to_sequences(train_data['text'])
maxlen = 100
x = pad_sequences(sequences, maxlen=maxlen)
y = train_data['label']

In [26]:
#划分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [34]:
#模型搭建
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(3,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'

In [ ]:
#训练模型
batch_size = 50
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
#评估
accuracy = model.evaluate(x_test, y_test, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))